In [ ]:
import pymysql
import pandas as pd
#DB연동
conn = pymysql.connect(host='localhost',
                       database='shakeit', # 자기 데이터 베이스에 맞게 접속
                       user='root',
                       passwd='mlb10200', #자기 비밀번호
                       charset='utf8',
                       local_infile=1) #혹시나 안되면 이거 켜서 접속

cursor = conn.cursor()

# member 테이블 조회 -> 데이터프레임에 저장
# like_table
# recruit

sql = "select * from shakeit.member"
cursor.execute(sql)
result1 = cursor.fetchall()
result1
sql = "select * from shakeit.like_table"
cursor.execute(sql)
result2 = cursor.fetchall()
result2
sql = "select * from shakeit.job"
cursor.execute(sql)
result3 = cursor.fetchall()
result3[0]

import pandas as pd
df1 = pd.DataFrame(result1)
df1.to_csv('shakeit_member.csv', encoding='utf8')
df2 = pd.DataFrame(result2)
df2.to_csv('shakeit_like.csv', encoding='utf8')
df3 = pd.DataFrame(result3)
df3.to_csv('shakeit_job.csv', encoding='utf8')

In [3]:
import pandas as pd
df1 = pd.read_csv('/Users/sky/class_python/shakeit_member.csv', index_col=0)
df1.rename(columns = {'0':'memberid'}, inplace=True)
#df1.drop(['1','2','3','4','5'], axis=1 ,inplace=True)
df1

,memberid,1,2,3,4,5
0,sky,123,123@naver.com,user,2021-12-08 13:09:08,0
1,sky2,123,123@naver.com,user,2021-12-09 00:56:29,0


In [33]:
df2 = pd.read_csv('/Users/sky/class_python/shakeit_like.csv', index_col=0)
a = pd.read_csv('/Users/sky/class_python/shakeit_like.csv', index_col=0)
#print(df2)
#a = a['1']
#df2['3'] = a
df2

,0,1,2
0,sky,8,2021-12-08
1,sky,133,2021-12-08
2,sky2,75,2021-12-09
3,sky2,4068,2021-12-09
4,sky2,9331,2021-12-09


In [5]:
df2.rename(columns = {'0':'memberid', '1':'like_Index', '3':'company_Index'}, inplace=True)
df2.drop(['2'], axis=1 ,inplace=True)
df2

,memberid,like_Index,company_Index
0,sky,8,8
1,sky,133,133
2,sky2,75,75
3,sky2,4068,4068
4,sky2,9331,9331


In [6]:
df3 = pd.read_csv('/Users/sky/class_python/shakeit_job.csv', index_col=0)
df3.drop(['2', '3', '4', '5', '6', '7', '8', '9', '10'], axis=1 ,inplace=True)
df3.rename(columns = {'0':'company_Index', '1':'company'}, inplace=True)
df3.head(1)

,company_Index,company
0,1,디휴브(DEHUV)


In [7]:
df3.shape

(47645, 2)

In [52]:
df3.head()

,company_Index,company
0,1,디휴브(DEHUV)
1,2,㈜레미유코스메틱
2,3,㈜순진
3,4,효성에프엠에스㈜
4,5,㈜빗썸코리아


In [85]:
company_member = pd.merge(df3["company_Index"], df1["memberid"], how="cross")

In [86]:
company_member["value"] = 0

In [87]:
company_member

,company_Index,memberid,value
0,1,sky,0
1,1,sky2,0
2,2,sky,0
3,2,sky2,0
4,3,sky,0
...,...,...,...
95285,47643,sky2,0
95286,47644,sky,0
95287,47644,sky2,0
95288,47645,sky,0


In [92]:
company_member.to_csv('company_member.csv', encoding='utf8')

In [34]:
df2

,0,1,2
0,sky,8,2021-12-08
1,sky,133,2021-12-08
2,sky2,75,2021-12-09
3,sky2,4068,2021-12-09
4,sky2,9331,2021-12-09


In [88]:
for row in df2.values:
    #print(row)
    filter = company_member.apply(lambda x: ((x[0] == row[1]) and (x[1] == row[0])), axis=1)
    # filter = company_member.apply(lambda x: print((x[0], x[1])), axis=1)
    #print(row)
    #filter =  company_member((company_member["memberid"] == row[0]) and (company_member["company_Index"] == row[1])) #(company_member["memberid"] == row[0])
    print(sum(filter))
    company_member["value"][filter] = 1
    

1


/var/folders/mh/q1r71q296q7grz41z51vbp6w0000gq/T/ipykernel_20174/1436067530.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_member["value"][filter] = 1


1
1
1
1


In [89]:
company_member.sum(axis=0)

company_Index                                           2270093670
memberid         skysky2skysky2skysky2skysky2skysky2skysky2skys...
value                                                            5
dtype: object

In [71]:
df2

,0,1,2
0,sky,8,2021-12-08
1,sky,133,2021-12-08
2,sky2,75,2021-12-09
3,sky2,4068,2021-12-09
4,sky2,9331,2021-12-09


In [90]:
company_member.iloc[14:16]

,company_Index,memberid,value
14,8,sky,1
15,8,sky2,0


In [93]:
prefer_table = company_member.pivot_table(index='memberid', columns='company_Index', values='value', aggfunc=np.sum)
prefer_table

company_Index,1,2,3,4,5,6,7,8,9,10,...,47636,47637,47638,47639,47640,47641,47642,47643,47644,47645
memberid,,,,,,,,,,,,,,,,,,,,,
sky,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
sky2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
